In [135]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 200)

In [136]:
#downloading initial file.
df_first=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_221008.txt',sep=',')
# second one to fill the empty rows after diff()
df_m1=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_221001.txt',sep=',')


In [137]:
#concatenating files for the initail and -1 week to get values for the first row after diff()
df=pd.concat([df_m1,df_first],axis=0,ignore_index=True)
df.reset_index(inplace=True,drop=True)

In [138]:
df.isnull().sum()

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64

No null values

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422384 entries, 0 to 422383
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count   Dtype 
---  ------                                                                --------------   ----- 
 0   C/A                                                                   422384 non-null  object
 1   UNIT                                                                  422384 non-null  object
 2   SCP                                                                   422384 non-null  object
 3   STATION                                                               422384 non-null  object
 4   LINENAME                                                              422384 non-null  object
 5   DIVISION                                                              422384 non-null  object
 6   DATE                                                                  422384 non-null  objec

All columns except DATE and TIME are in the correct format. Let's unite that columns and change to datetime.

In [140]:
# snake_case column names
df.columns=df.columns.str.lower().str.replace(" ","")
# converting date to datetime format
df["datetime"]=df["date"]+" "+df["time"]
df.datetime=pd.to_datetime(df["datetime"])
df.drop(columns=["date","time"],inplace=True)



In [141]:
df.head(2)

,c/a,unit,scp,station,linename,division,desc,entries,exits,datetime
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767632,2742700,2022-09-24 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767638,2742703,2022-09-24 04:00:00


Now features are fine.

----

This code creates multindex that will help group dataframe and calculate absolute values for entries and exits instead of relevant.

In [142]:
df['stat_id']=df["station"]+" "+df["scp"]+" "+df["c/a"]
df=df.set_index(["stat_id","datetime"]).sort_index()

df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 04:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 08:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 12:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 09:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 13:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 17:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 21:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2022-09-24 00:00:00        L      BMT  REGULAR   
                        2022-09-24 04:00:00        L      BMT  REGULAR   
                        2022-09-24 08:00:00        L      BMT  REGULAR   
                        2022-09-24 12:00:00        L      BMT  REGULAR   
                        2022-09-24 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        6      IRT  REGULAR   
                        2022-10-07 09:00:00        6      IRT  REGULAR   
                        2022-10-07 13:00:00        6      IRT  REGULAR   
                        2022-10-07 17:00:00        6      IRT  REGULAR   
                        2022-10-07 21:00:00        6      IRT  REGULAR   

                                              entries     exits  
stat_id                 datetime                                 
1 AV 00-00-00 H007      2022-09-24 00:00:00  15841287  19006318  
                        2022-09-24 04:00:00  15841308  19006484  
                        2022-09-24 08:00:00  15841319  19006685  
                        2022-09-24 12:00:00  15841363  19007161  
                        2022-09-24 16:00:00  15841455  19007876  
...                                               ...       ...  
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        39       289  
                        2022-10-07 09:00:00        39       289  
                        2022-10-07 13:00:00        39       289  
                        2022-10-07 17:00:00        39       289  
                        2022-10-07 21:00:00        39       289  

[422384 rows x 9 columns]

In [144]:
df['entries_abs']=df.groupby(level=0)['entries'].diff()
df['exits_abs']=df.groupby(level=0)['exits'].diff()

In [145]:
df[df["entries_abs"]<0].sample(20)

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,
72 ST 00-00-03 R161B,2022-09-30 17:00:00,R161B,R452,00-00-03,72 ST,123,IRT,REGULAR,689172242,1760469209,-237.0,302.0
DEKALB AV 00-00-01 H023,2022-09-25 12:00:00,H023,R236,00-00-01,DEKALB AV,L,BMT,REGULAR,1136400484,490093351,-55.0,19.0
JAMAICA 179 ST 01-03-02 N342,2022-10-02 04:00:00,N342,R019,01-03-02,JAMAICA 179 ST,F,IND,REGULAR,1290944509,309364173,-5.0,12.0
167 ST 00-00-00 N207,2022-09-30 08:00:00,N207,R104,00-00-00,167 ST,BD,IND,REGULAR,1827447874,1829878208,-305.0,116.0
DEKALB AV 00-00-00 C008,2022-09-27 04:00:00,C008,R099,00-00-00,DEKALB AV,BDNQR,BMT,REGULAR,1999237231,1851589446,-9.0,31.0
135 ST 00-00-00 R306,2022-10-02 04:00:00,R306,R207,00-00-00,135 ST,23,IRT,REGULAR,855175309,923065911,-7.0,35.0
86 ST 01-00-05 R249,2022-10-06 00:00:00,R249,R179,01-00-05,86 ST,456,IRT,REGULAR,1073638049,408824821,-35.0,59.0
66 ST-LINCOLN 00-00-00 R160A,2022-10-06 17:00:00,R160A,R164,00-00-00,66 ST-LINCOLN,1,IRT,REGULAR,28,30,-870748.0,-1274424.0
23 ST 00-00-02 N508,2022-10-02 00:00:00,N508,R453,00-00-02,23 ST,FM,IND,REGULAR,1824075576,1932225498,-77.0,46.0


In [106]:
drop_rows_condition = (df["entries_abs"] < 0)
df_cleaned = df[ ~drop_rows_condition ]

Now, let's drop entries, exits and unnecessary indexes.

In [399]:
pp=df.copy()
pp

c/a  unit       scp    station linename division  \
station   scp                                                                  
1 AV      00-00-00 32771   H007  R248  00-00-00       1 AV        L      BMT   
                   32772   H007  R248  00-00-00       1 AV        L      BMT   
                   32773   H007  R248  00-00-00       1 AV        L      BMT   
                   32774   H007  R248  00-00-00       1 AV        L      BMT   
                   32775   H007  R248  00-00-00       1 AV        L      BMT   
...                         ...   ...       ...        ...      ...      ...   
ZEREGA AV 00-05-01 185405  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185406  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185407  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185408  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185409   NaN   NaN       NaN        NaN      NaN      NaN   

                              desc     entries       exits  \
station   scp                                                
1 AV      00-00-00 32771   REGULAR  15841287.0  19006318.0   
                   32772   REGULAR  15841308.0  19006484.0   
                   32773   REGULAR  15841319.0  19006685.0   
                   32774   REGULAR  15841363.0  19007161.0   
                   32775   REGULAR  15841455.0  19007876.0   
...                            ...         ...         ...   
ZEREGA AV 00-05-01 185405  REGULAR        39.0       289.0   
                   185406  REGULAR        39.0       289.0   
                   185407  REGULAR        39.0       289.0   
                   185408  REGULAR        39.0       289.0   
                   185409      NaN         NaN         NaN   

                                     datetime  entries_abs  exits_abs  
station   scp                                                          
1 AV      00-00-00 32771  2022-09-24 00:00:00          NaN        NaN  
                   32772  2022-09-24 04:00:00         21.0      166.0  
                   32773  2022-09-24 08:00:00         11.0      201.0  
                   32774  2022-09-24 12:00:00         44.0      476.0  
                   32775  2022-09-24 16:00:00         92.0      715.0  
...                                       ...          ...        ...  
ZEREGA AV 00-05-01 185405 2022-10-07 05:00:00          0.0        0.0  
                   185406 2022-10-07 09:00:00          0.0        0.0  
                   185407 2022-10-07 13:00:00          0.0        0.0  
                   185408 2022-10-07 17:00:00          0.0        0.0  
                   185409                 NaT          NaN        NaN  

[422384 rows x 12 columns]

In [400]:
pp.drop(columns=["entries","exits"],inplace=True)
pp

c/a  unit       scp    station linename division  \
station   scp                                                                  
1 AV      00-00-00 32771   H007  R248  00-00-00       1 AV        L      BMT   
                   32772   H007  R248  00-00-00       1 AV        L      BMT   
                   32773   H007  R248  00-00-00       1 AV        L      BMT   
                   32774   H007  R248  00-00-00       1 AV        L      BMT   
                   32775   H007  R248  00-00-00       1 AV        L      BMT   
...                         ...   ...       ...        ...      ...      ...   
ZEREGA AV 00-05-01 185405  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185406  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185407  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185408  R419  R326  00-05-01  ZEREGA AV        6      IRT   
                   185409   NaN   NaN       NaN        NaN      NaN      NaN   

                              desc            datetime  entries_abs  exits_abs  
station   scp                                                                   
1 AV      00-00-00 32771   REGULAR 2022-09-24 00:00:00          NaN        NaN  
                   32772   REGULAR 2022-09-24 04:00:00         21.0      166.0  
                   32773   REGULAR 2022-09-24 08:00:00         11.0      201.0  
                   32774   REGULAR 2022-09-24 12:00:00         44.0      476.0  
                   32775   REGULAR 2022-09-24 16:00:00         92.0      715.0  
...                            ...                 ...          ...        ...  
ZEREGA AV 00-05-01 185405  REGULAR 2022-10-07 05:00:00          0.0        0.0  
                   185406  REGULAR 2022-10-07 09:00:00          0.0        0.0  
                   185407  REGULAR 2022-10-07 13:00:00          0.0        0.0  
                   185408  REGULAR 2022-10-07 17:00:00          0.0        0.0  
                   185409      NaN                 NaT          NaN        NaN  

[422384 rows x 10 columns]

In [401]:
pp.set_index(pp['datetime'],drop=True,inplace=True)
pp.drop(columns='datetime',inplace=True)
pp

,c/a,unit,scp,station,linename,division,desc,entries_abs,exits_abs
datetime,,,,,,,,,
2022-09-24 00:00:00,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,NaN,NaN
2022-09-24 04:00:00,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,21.0,166.0
2022-09-24 08:00:00,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,11.0,201.0
2022-09-24 12:00:00,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,44.0,476.0
2022-09-24 16:00:00,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,92.0,715.0
...,...,...,...,...,...,...,...,...,...
2022-10-07 05:00:00,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,0.0,0.0
2022-10-07 09:00:00,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,0.0,0.0
2022-10-07 13:00:00,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,0.0,0.0


In [415]:
df_first.shape

(211003, 11)

In [402]:
pp=pp.loc["2022-10-01":"2022-10-07"]


In [433]:
pp[pp["entries_abs"]<0]

,c/a,unit,scp,station,linename,division,desc,entries_abs,exits_abs
datetime,,,,,,,,,
2022-10-01 01:00:00,R252,R180,00-00-00,103 ST,6,IRT,REGULAR,-1.378591e+07,-10435843.0
2022-10-01 01:00:00,R170,R191,00-00-00,103 ST,1,IRT,REGULAR,-1.372576e+06,-1699198.0
2022-10-01 09:00:00,R170,R191,00-00-00,103 ST,1,IRT,REGULAR,-1.515846e+07,-12135010.0
2022-10-01 13:00:00,R170,R191,00-00-00,103 ST,1,IRT,REGULAR,-1.372495e+06,-1699375.0
2022-10-01 17:00:00,R252,R180,00-00-00,103 ST,6,IRT,REGULAR,-1.378564e+07,-10435298.0
...,...,...,...,...,...,...,...,...,...
2022-10-07 05:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,-4.000000e+00,16.0
2022-10-07 09:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,-2.470000e+02,91.0
2022-10-07 13:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,-1.230000e+02,127.0


station load
Most loaded and most empty stations
plot it
amount of persons per day for line/all subway
load per day for the whole system
